In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import json

In [4]:
anno_dir = '/DataMount/xl260/Casey_aapm2024/WSI_Output/json/700837.json'

In [7]:
def load_json(name):
    # load the json file (may take ~20 secs)
    json_path_wsi = name

    bbox_list_wsi = []
    centroid_list_wsi = []
    contour_list_wsi = [] 
    type_list_wsi = []

    # add results to individual lists
    with open(json_path_wsi) as json_file:
        data = json.load(json_file)
        mag_info = data['mag']
        nuc_info = data['nuc']
        for inst in nuc_info:
            inst_info = nuc_info[inst]
            inst_centroid = inst_info['centroid']
            centroid_list_wsi.append(inst_centroid)
            inst_contour = inst_info['contour']
            contour_list_wsi.append(inst_contour)
            inst_bbox = inst_info['bbox']
            bbox_list_wsi.append(inst_bbox)
            inst_type = inst_info['type']
            type_list_wsi.append(inst_type)
            
    return centroid_list_wsi, contour_list_wsi, bbox_list_wsi, type_list_wsi

In [ ]:
# load the json file
centroid_list_wsi, contour_list_wsi, bbox_list_wsi, type_list_wsi = load_json(anno_dir)
# convert json file to Qupath format
centroid_list_qupath = []
contour_list_qupath = []
bbox_list_qupath = []
type_list_qupath = []
for i in range(len(centroid_list_wsi)):
    inst_centroid = centroid_list_wsi[i]
    inst_contour = contour_list_wsi[i]
    inst_bbox = bbox_list_wsi[i]
    inst_type = type_list_wsi[i]
    inst_qupath = {'X': inst_centroid[0], 'Y': inst_centroid[1], 'class': inst_type}
    centroid_list_qupath.append(inst_qupath)
    inst_qupath = []
    for point in inst_contour:
        inst_qupath.append({'X': point[0], 'Y': point[1]})
    contour_list_qupath.append(inst_qupath)
    inst_qupath = {'X': inst_bbox[0], 'Y': inst_bbox[1], 'width': inst_bbox[2], 'height': inst_bbox[3]}
    bbox_list_qupath.append(inst_qupath)
    type_list_qupath.append(inst_type)

# save the json file
qupath_json = {'annotations': []}
for i in range(len(centroid_list_qupath)):
    inst_annotation = {'centroid': centroid_list_qupath[i], 'contour': contour_list_qupath[i], 'bbox': bbox_list_qupath[i], 'type': type_list_qupath[i]}
    qupath_json['annotations'].append(inst_annotation)
qupath_json_str = json.dumps(qupath_json, indent=4)
qupath_json_path = anno_dir.replace('.json', '_qupath.json')
with open(qupath_json_path, 'w') as json_file:
    json_file.write(qupath_json_str)
print('Qupath json file saved to:', qupath_json_path)

